In [1]:
import pandas as pd
import numpy as np
import json
import yaml
from scipy import stats
import seaborn as sn
import pickle

import os
from statsmodels.stats.multitest import multipletests
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
from scipy.stats import rankdata

CSV_DIR = 'csv_merged'

In [2]:
def parser_method_dict(df, batch = 16):
    ''''This function parse the dict and then compute the AUC per image
        It returns a dataframe of the shape (Nb_Batch, Nb_images_per_bach)
        np.trapz integrates the function under the curve
    '''
    dataf = pd.DataFrame(columns = [f"{i}" for i in range(batch)])
    for i in range(df.shape[0]):
        row = yaml.safe_load(df.iloc[i].iloc[0])
        dataf.loc[i] = [np.trapz(row[j]) for j in row]
    #print(dataf)
    return dataf

In [3]:
def parser_method_dict_with_layers(df, batch = 16):
    ''''This function parse the dict and then compute the AUC per image
        It returns a dataframe of the shape (Nb_Batch, Nb_images_per_bach)
        np.trapz integrates the function under the curve
        Note that df.iloc[0] has a dic e.g., {"layer1": [list of corr of bacth], "layer2": [list of corr of batch]}
    '''
    dataf = pd.DataFrame(columns = [f"{i}" for i in range(batch)])
    for i in range(df.shape[0]): # loop over the number of batches
        row = yaml.safe_load(df.iloc[i].iloc[0]) # Get the batch result
        fillna = lambda v: 0.0 if v == 'nan' else v
        # p is the index of image, j is the layer_name, np.trapz computes the AUC
        remerged_batch = []
        pos = -1
        try:
            dataf.loc[i] = [np.trapz([fillna(row[j][p]) for j in row]) for p in range(batch)]
        except:
            last_row = yaml.safe_load(df.iloc[-1].iloc[0])
            remerged_batch = [np.trapz([fillna(row[j][p]) for j in row]) for p in range(8)] + [np.trapz([fillna(last_row[j][p]) for j in row]) for p in range(8)]
            if pos == -1:
                pos = i
        if remerged_batch:
            dataf.loc[pos] = remerged_batch
            dataf = dataf[:-1]
    return dataf

In [4]:
metrics =  [
    'Model Parameter Randomisation',
    'Monotonicity Nguyen',
    'Local Lipschitz Estimate',
    'Faithfulness Estimate',
    'Faithfulness Correlation',
    'Avg-Sensitivity',
    'Random Logit',
    'Max-Sensitivity',
    'Sparseness', 
    'EffectiveComplexity',
    'Monotonicity Arya',
    'Complexity',
    'Pixel-Flipping',
    'Selectivity'    
]
                # ['SensitivityN': problem with implementation,
                #'Region Perturbation' seems to be same as region perturbation, 
                #'Continuity Test': Difficult to aggregate result, the paper just plot it
                #'Completeness' always returns False]
                # Nonsentitivity is removed

metrics_with_different_baselines = {
    'Faithfulness Estimate',
    'Faithfulness Correlation',
    'Monotonicity Arya',
    'Monotonicity Nguyen',
    'Pixel-Flipping',
    'Selectivity',
}
                
baselines = [
    'baseline_black',
    # 'baseline_mean', not used anymore as there's some probs with quantus implementation
    'baseline_random',
    'baseline_uniform',
    'baseline_white'
]

transform = {
    'Monotonicity Nguyen': lambda x: x.fillna(x.mean()),
    'Local Lipschitz Estimate': lambda x: -x, 
    'Faithfulness Estimate': lambda x: x.fillna(x.mean()),
    'Faithfulness Correlation': lambda x: x.fillna(x.mean()),
    'Avg-Sensitivity': lambda x: -x,
    'Random Logit': lambda x: x.fillna(x.mean()),
    'Sparseness': lambda x: x.fillna(x.mean()),
    'EffectiveComplexity': lambda x: -x,
    'Nonsensitivity': lambda x: -x,
    'Pixel-Flipping': lambda x: x.apply(lambda row: - np.trapz(row), axis=1),
    'Max-Sensitivity': lambda x: -x,
    'Complexity': lambda x: -x.fillna(x.mean()),
    'Selectivity': lambda x: -parser_method_dict(x),
    'Model Parameter Randomisation': lambda x: -parser_method_dict_with_layers(x),
    'Monotonicity Arya': lambda x: x,
}

aggregate = {
    'Monotonicity Nguyen': lambda x: np.tanh(np.mean(np.arctanh(x))),
    'Local Lipschitz Estimate': np.mean,
    'Faithfulness Estimate': lambda x: np.tanh(np.mean(np.arctanh(x))),
    'Faithfulness Correlation': lambda x: np.tanh(np.mean(np.arctanh(x))),
    'Avg-Sensitivity': np.mean,
    'Random Logit': np.mean,
    'Max-Sensitivity': np.mean,
    'Sparseness': np.mean, 
    'EffectiveComplexity': np.mean,
    'Monotonicity Arya': np.mean,
    'Complexity': np.mean,
    'Pixel-Flipping': np.mean,
    'Selectivity': np.mean,
    'Model Parameter Randomisation': np.mean
}

methods = ['integratedgrad', 'smoothgrad', 'guidedbackprop', 'rise', 'gradcam',
           'scorecam', 'layercam', 'random', 'sobel', 'gaussian', 'polycam',
           'cameras']#, 'extremal_perturbation']

models = ['resnet50']
datasets = ['cifar10']

dico_ranks = {}

In [22]:
for dataset in datasets:
    print(f'DATASET {dataset.upper()}')
    dico_ranks[dataset] = {}
    for model in models:
        print(f'MODEL {model.upper()}')
        dico_ranks[dataset][model] = {}
        for metr in metrics:
            if metr in metrics_with_different_baselines and model == 'resnet50' and dataset == 'imagenet':
                for baseline in baselines:
                    metr_with_baseline = f'{metr} with {baseline}'
                    print(f"-- Metric: {metr_with_baseline}")
                    data = pd.DataFrame()
                    for meth in methods:
                        if meth == 'cameras' or meth == 'extremal_perturbation':
                            csv_name = f"{CSV_DIR}/{meth}_{model}_{dataset}_{metr}.csv"
                        else:
                            csv_name = f"{CSV_DIR}/{meth}_{model}_{dataset}_{metr}_{baseline}.csv"
                        df = pd.read_csv(csv_name, header = None)
                        data[meth] = transform[metr](df).values.flatten()[:2000]
                        scores.append(-aggregate[metr](data))
                    ranks = np.array([rankdata(-p) for p in data.values])
                    average_ranks = np.mean(ranks, axis=0)
                    dico_ranks[dataset][model][metr_with_baseline] = average_ranks
            else:
                print(f"-- Metric: {metr}")
                data = pd.DataFrame()
                for meth in methods:
                    csv_name = f"{CSV_DIR}/{meth}_{model}_{dataset}_{metr}.csv"
                    df = pd.read_csv(csv_name, header = None)
                    try:
                        data[meth] = transform[metr](df).values.flatten()[:2000]
                    except:
                        data[meth] = transform[metr](df).values.flatten()
                ranks = np.array([rankdata(-p) for p in data.values])
                average_ranks = np.mean(ranks, axis=0)
                dico_ranks[dataset][model][metr] = average_ranks
        print()
    print()
    print()

DATASET CIFAR10
MODEL RESNET50
-- Metric: Model Parameter Randomisation


ValueError: Length of values (1984) does not match length of index (2000)

In [24]:
temp = transform[metr](df).values.flatten()

In [25]:
temp.shape

(1984,)

In [ ]:
with open('rankings_by_rank_aggreg_cifar10.pickle', 'wb') as file:
    pickle.dump(dico_ranks, file)

In [ ]:
with open('rankings_by_rank_aggreg_cifar10.pickle', 'rb') as file:
    dico_ranks = pickle.load(file)

In [ ]:
def compute_kendall_w(dico_ranks, metrics):
    rankings = [rankdata(dico_ranks[metric]) for metric in metrics]
    data = []
    for i in range(len(rankings[0])):
        data.append([rankings[j][i] for j in range(len(rankings))])   
    
    # source of the implementation : https://github.com/ugolbck/kendall-w/blob/master/kendall_w/kendall_w.py
    m = len(data[0])
    n = len(data)
    sums = [sum(x) for x in data]
    Rbar = sum(sums) / n
    S = sum([(sums[x] - Rbar) ** 2 for x in range(n)])
    W = (12 * S) / (m ** 2 * (n ** 3 - n))

    return W

In [ ]:
def compute_kendall_tau(dico_ranks, metrics):
    tau_values = []
    p_values = []

    for metric_a in metrics:
        current_tau_values = []
        current_p_values = []
        for metric_b in metrics:
            tau, p_value = stats.kendalltau(dico_ranks[metric_a], dico_ranks[metric_b])
            current_tau_values.append(tau)
            current_p_values.append(p_value)
        tau_values.append(current_tau_values)
        p_values.append(current_p_values)
        
    return tau_values, p_values

In [ ]:
def plot_corr_matrix_figure(dico_ranks, metrics, y_labels, x_labels, filename, fig_size, rotate_x=True, half_rotate_x=False, rotate_y=True, subgroups=None):
    tau_values, p_values = compute_kendall_tau(dico_ranks, metrics)
    
    p_values_flattened = [p_value for i, sublist in enumerate(p_values) for p_value in sublist[:i]]
    reject, _, _, _ = multipletests(p_values_flattened, alpha=0.05, method='holm')

    sn.set(rc={'figure.figsize': fig_size})

    p_values_flattened = [p_value for i, sublist in enumerate(p_values) for p_value in sublist[:i]]
    reject, _, _, _ = multipletests(p_values_flattened, alpha=0.05, method='holm')
    mask = np.ones((len(metrics),len(metrics)), dtype=bool)
    current_post = 0
    for i in range(len(metrics)):
        for j in range(i):
            mask[i][j] = reject[current_post]
            mask[j][i] = reject[current_post]
            current_post += 1

    sn.heatmap(tau_values,
               annot=True,
               vmin=-1,
               vmax=1,
               cbar=False,
               xticklabels=x_labels,
               yticklabels=y_labels,
               mask=mask,
               cmap='viridis')

    mask = np.ones((len(metrics),len(metrics)), dtype=bool)
    current_post = 0
    for i in range(len(metrics)):
        for j in range(i):
            mask[i][j] = not reject[current_post]
            mask[j][i] = not reject[current_post]
            current_post += 1
        mask[i][i] = False

    ax = sn.heatmap(tau_values,
                    annot=True,
                    annot_kws={"style": "italic", "weight": "bold"},
                    vmin=-1,
                    vmax=1,
                    cbar=False,
                    xticklabels=x_labels,
                    yticklabels=y_labels,
                    mask=mask,
                    cmap='viridis')

    if half_rotate_x:
        plt.xticks(rotation=20, ha="right")
    elif rotate_x:
        plt.xticks(rotation=0)
    if rotate_y:
        plt.yticks(rotation=0)
    
    start_i = 0
    if subgroups:
        for subgroup_size in subgroups:
            ax.add_patch(Rectangle((start_i, start_i), subgroup_size, subgroup_size, fill=False, edgecolor='crimson', lw=4, clip_on=False))
            start_i += subgroup_size
    
    plt.savefig(f'./results/{filename}.eps', bbox_inches='tight', format='eps')
    plt.show()
    plt.close()
    
    print('RESULTS')
    print(f"Kendall's W: {compute_kendall_w(dico_ranks, metrics)}")
    print("Kendall's Tau:")
    for i, metric_a in enumerate(metrics):
        for j, metric_b in enumerate(metrics):
            if i < j:
                print(f'{metric_a} / {metric_b} : {tau_values[i][j]} ({p_values[i][j]})')

# Figures and results for article

## ResNet50 (Cifar10)

In [ ]:
df_ranks = pd.DataFrame(dico_ranks['cifar10']['resnet50'], index= methods)
metrics_with_baselines = sorted(dico_ranks.keys())

In [ ]:
df_ranks

### Faithfulness with black baseline and different metrics

In [ ]:
selected_metrics = ['Faithfulness Correlation','Faithfulness Estimate','Monotonicity Arya','Monotonicity Nguyen','Pixel-Flipping','Selectivity']
labels = ['Faithfulness Correlation (FC)','Faithfulness Estimate (FE)','Monotonicity Arya (MA)','Monotonicity Nguyen (MN)','Pixel-Flipping (PF)','Selectivity (Se)']
short_labels = ['FC','FE', 'MA','MN','PF','Se']
filename = 'rebuttal_final_corr_matrix_rank_aggreg_holm_corr_faithfulness_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, short_labels, short_labels, filename, fig_size=(5,3))

### Complexity

In [ ]:
selected_metrics = ['Complexity','EffectiveComplexity','Sparseness']
labels = ['Complexity (C)', 'Effective Complexity (E)', 'Sparseness (S)']
short_labels = ['C','E', 'S']
filename = 'final_corr_matrix_rank_aggreg_holm_corr_complexity_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, short_labels, short_labels, filename, fig_size=(2,1.5))

### Randomization

In [ ]:
selected_metrics = ['Model Parameter Randomisation', 'Random Logit']
labels = ['Model Parameter Randomisation (M)', 'Random Logit (R)']
short_labels = ['M','R']
filename = 'final_corr_matrix_rank_aggreg_holm_corr_randomization_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, short_labels, short_labels, filename, fig_size=(1.5,1))

## Robustness

In [ ]:
selected_metrics = ['Avg-Sensitivity','Local Lipschitz Estimate', 'Max-Sensitivity']
labels = ['Avg-Sensitivity (A)', 'Local Lipschitz Estimate (L)', 'Max-Sensitivity (M)']
short_labels = ['A','L', 'M']
filename = 'final_corr_matrix_rank_aggreg_holm_corr_robustness_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, short_labels, short_labels, filename, fig_size=(2,1.5))

## All metrics with default baselines (i.e. black)

In [ ]:
selected_metrics =  [
    'Complexity',
    'EffectiveComplexity',
    'Sparseness',
    'Model Parameter Randomisation',
    'Random Logit',   
    'Avg-Sensitivity',
    'Local Lipschitz Estimate',
    'Max-Sensitivity',
    'Faithfulness Correlation',
    'Faithfulness Estimate',
    'Monotonicity Arya',
    'Monotonicity Nguyen',
    'Pixel-Flipping',
    'Selectivity'    
]

labels = [
    'Complexity',
    'EffectiveComplexity',
    'Sparseness',
    'Model Parameter Randomisation',
    'Random Logit',   
    'Avg-Sensitivity',
    'Local Lipschitz Estimate',
    'Max-Sensitivity',
    'Faithfulness Correlation',
    'Faithfulness Estimate',
    'Monotonicity Arya',
    'Monotonicity Nguyen',
    'Pixel-Flipping',
    'Selectivity'    
]

#for metric in metrics:
#    if metric in metrics_with_different_baselines:
#        selected_metrics.append(f'{metric} with baseline_black')
#    elif metr == 'Model Parameter Randomisation':
#        metr_with_baseline = f'{metr} with bottom_up'
#        final_dico_ranks[metr] = dico_ranks[metr_with_baseline]
#    else:
#        selected_metrics.append(metric)
#    labels.append(metric)

# selected_metrics.sort()
# labels.sort()
filename = 'final_corr_matrix_rank_aggreg_holm_corr_all_metrics_default_baselines_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, labels, labels, filename, fig_size=(10,8), rotate_x=False)

In [ ]:
filename = 'final_corr_matrix_rank_aggreg_holm_corr_all_metrics_default_baselines_with_subgroups_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, labels, labels, filename, fig_size=(10,8), half_rotate_x=True, subgroups=(3,2,3,6))

### All metrics with all baselines

In [ ]:
selected_metrics = sorted(dico_ranks['cifar10']['resnet50'].keys())
filename = 'final_corr_matrix_rank_aggreg_holm_corr_all_metrics_all_baselines_resnet50_cifar10'
plot_corr_matrix_figure(dico_ranks['cifar10']['resnet50'], selected_metrics, selected_metrics, selected_metrics, filename, fig_size=(23.4,16.54), rotate_x=False)